In [ ]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 45.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=3c5070952aa12513c34ee4dc2ecc3e83401a0d3e1b683fc0b90d606486324980
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MLlib").getOrCreate()
spark

In [ ]:
df = spark.read.csv("titanic.csv", header=True, inferSchema=True)
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
df.show(truncate=False)

+-----------+--------+------+-------------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                                   |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+-------------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                                |male  |22.0|1    |0    |A/5 21171       |7.25   |null |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)    |female|38.0|1    |0    |PC 17599        |71.2833|C85  |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                                 |female|26.0|0    |0    |STON/O2. 3101282|7.925  |null |S       |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath (Lily May Peel)           |female|35.0|1  

In [ ]:
df.describe().show(vertical=True)

-RECORD 0---------------------------
 summary     | count                
 PassengerId | 891                  
 Survived    | 891                  
 Pclass      | 891                  
 Name        | 891                  
 Sex         | 891                  
 Age         | 714                  
 SibSp       | 891                  
 Parch       | 891                  
 Ticket      | 891                  
 Fare        | 891                  
 Cabin       | 204                  
 Embarked    | 889                  
-RECORD 1---------------------------
 summary     | mean                 
 PassengerId | 446.0                
 Survived    | 0.3838383838383838   
 Pclass      | 2.308641975308642    
 Name        | null                 
 Sex         | null                 
 Age         | 29.69911764705882    
 SibSp       | 0.5230078563411896   
 Parch       | 0.38159371492704824  
 Ticket      | 260318.54916792738   
 Fare        | 32.2042079685746     
 Cabin       | null                 
 

In [ ]:
df.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
Pclass,712,2.240168539325843,0.8368543166903446,1,3
Sex,712,None,None,female,male
Age,712,29.64209269662921,14.49293290032352,0.42,80.0
SibSp,712,0.5140449438202247,0.9306921267673427,0,5
Parch,712,0.43258426966292135,0.8541814457454133,0,6
Fare,712,34.56725140449432,52.938648174710906,0.0,512.3292
Embarked,712,None,None,C,S
Survived,712,0.4044943820224719,0.4911389472541192,0,1


In [ ]:
shape = (df.count(), len(df.columns))
print(shape)

(891, 12)


In [ ]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [ ]:
df.groupBy("Survived").count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [ ]:
df = df.select(['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked','Survived'])

df.show()

+------+------+----+-----+-----+-------+--------+--------+
|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|Survived|
+------+------+----+-----+-----+-------+--------+--------+
|     3|  male|22.0|    1|    0|   7.25|       S|       0|
|     1|female|38.0|    1|    0|71.2833|       C|       1|
|     3|female|26.0|    0|    0|  7.925|       S|       1|
|     1|female|35.0|    1|    0|   53.1|       S|       1|
|     3|  male|35.0|    0|    0|   8.05|       S|       0|
|     3|  male|null|    0|    0| 8.4583|       Q|       0|
|     1|  male|54.0|    0|    0|51.8625|       S|       0|
|     3|  male| 2.0|    3|    1| 21.075|       S|       0|
|     3|female|27.0|    0|    2|11.1333|       S|       1|
|     2|female|14.0|    1|    0|30.0708|       C|       1|
|     3|female| 4.0|    1|    1|   16.7|       S|       1|
|     1|female|58.0|    0|    0|  26.55|       S|       1|
|     3|  male|20.0|    0|    0|   8.05|       S|       0|
|     3|  male|39.0|    1|    5| 31.275|       S|       

In [ ]:
for i in df.columns:
  print(i + ":" , df[df[i].isNull()].count())

Pclass: 0
Sex: 0
Age: 177
SibSp: 0
Parch: 0
Fare: 0
Embarked: 2
Survived: 0


In [ ]:
df = df.na.drop()

In [ ]:
for col in df.columns:
  print(col + ":" , df[df[col].isNull()].count())

Pclass: 0
Sex: 0
Age: 0
SibSp: 0
Parch: 0
Fare: 0
Embarked: 0
Survived: 0


In [ ]:
df.show(5)

+------+------+----+-----+-----+-------+--------+--------+
|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|Survived|
+------+------+----+-----+-----+-------+--------+--------+
|     3|  male|22.0|    1|    0|   7.25|       S|       0|
|     1|female|38.0|    1|    0|71.2833|       C|       1|
|     3|female|26.0|    0|    0|  7.925|       S|       1|
|     1|female|35.0|    1|    0|   53.1|       S|       1|
|     3|  male|35.0|    0|    0|   8.05|       S|       0|
+------+------+----+-----+-----+-------+--------+--------+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder

gender_indexer = StringIndexer(inputCol="Sex", outputCol="Sexindex")
gender_encoder = OneHotEncoder(inputCol="Sexindex", outputCol="Sexvec")

embark_indexer = StringIndexer(inputCol="Embarked", outputCol="Embarkedindex")
embark_encoder = OneHotEncoder(inputCol="Embarkedindex", outputCol="Embarkedvec")

In [ ]:
assembler = VectorAssembler(inputCols=["Pclass","Sexvec","Age","SibSp","Parch","Fare","Embarkedvec"],
                            outputCol="features")


In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
lr_titanic = LogisticRegression(featuresCol="features", labelCol="Survived")

In [ ]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                            gender_encoder,embark_encoder,
                            assembler,lr_titanic])

In [ ]:
train_data, test_data = df.randomSplit([0.7,0.3])

In [ ]:
fit_model = pipeline.fit(train_data)

In [ ]:
result = fit_model.transform(test_data)

In [ ]:
result.show(truncate=10)

+------+------+----+-----+-----+--------+--------+--------+--------+-------------+---------+-----------+----------+-------------+-----------+----------+
|Pclass|   Sex| Age|SibSp|Parch|    Fare|Embarked|Survived|Sexindex|Embarkedindex|   Sexvec|Embarkedvec|  features|rawPrediction|probability|prediction|
+------+------+----+-----+-----+--------+--------+--------+--------+-------------+---------+-----------+----------+-------------+-----------+----------+
|     1|female|14.0|    1|    2|   120.0|       S|       1|     1.0|          0.0|(1,[],[])| (2,[0],...|[1.0,0....|   [-3.000...| [0.0473...|       1.0|
|     1|female|16.0|    0|    1|    39.4|       S|       1|     1.0|          0.0|(1,[],[])| (2,[0],...|[1.0,0....|   [-3.423...| [0.0315...|       1.0|
|     1|female|16.0|    0|    1| 57.9792|       C|       1|     1.0|          1.0|(1,[],[])| (2,[1],...|[1.0,0....|   [-4.053...| [0.0170...|       1.0|
|     1|female|19.0|    0|    0|    30.0|       S|       1|     1.0|          0.0|

In [ ]:
result.select(["prediction","Survived"]).show(50)

+----------+--------+
|prediction|Survived|
+----------+--------+
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       1|
|       1.0|       0|
|       1.0|       0|
|       1.0|       0|
|       1.0|       1|
|       1.0|       0|
|       0.0|       0|
|       1.0|       0|
|       0.0|       0|
|       0.0|       1|
|       1.0|       0|
|       1.0|       1|
|       1.0|       1|
|       0.0|       0|
|       0.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="Survived")


BinaryClassificationEvaluator_bf4b52e77e9f

In [ ]:
eval = my_eval.evaluate(result)

In [ ]:
eval

0.8546296296296304